In [1]:
from orbax.checkpoint import PyTreeCheckpointer

import _pickle as pickle
import jax
import json
import logging
import os
import timeit

from jaxl.buffers import get_buffer
from jaxl.constants import CONST_DEFAULT, CONST_MODEL, CONST_OBS_RMS, CONST_POLICY
from jaxl.models import (
    get_model,
    get_policy,
    policy_output_dim,
)
from jaxl.envs import get_environment
from jaxl.envs.rollouts import EvaluationRollout
from jaxl.utils import set_seed

In [1]:
rl_trained_path = "/Users/chanb/research/personal/jaxl/jaxl/logs/inverted_pendulum/reinforce/05-30-23_11_06_49-d9afde67-e3fd-4d7c-8df9-4be90167d3a3"
bc_trained_path = "/Users/chanb/research/personal/jaxl/jaxl/logs/inverted_pendulum/behavioural_cloning/05-30-23_11_25_47-215b4371-75f6-4bc8-88a4-498733c1594f"